In [1]:
from edgedroid.models.timings import *

import numpy as np
import numpy.typing as npt
# import pandas as pd

runs_per_model = 30
task_steps = 100


models = {
    "empirical": {
        "low": EmpiricalExecutionTimeModel(neuroticism=0.0),
        "high": EmpiricalExecutionTimeModel(neuroticism=1.0),
    },
    "theoretical": {
        "low": TheoreticalExecutionTimeModel(neuroticism=0.0),
        "high": TheoreticalExecutionTimeModel(neuroticism=1.0),
    },
    "rolling-ttf": {
        "low": ExpKernelRollingTTFETModel(neuroticism=0.0),
        "high": ExpKernelRollingTTFETModel(neuroticism=1.0),
    },
    "fitted-rolling-ttf": {
        "low": DistExpKernelRollingTTFETModel(neuroticism=0.0),
        "high": DistExpKernelRollingTTFETModel(neuroticism=1.0),
    },
    "naive": {"n/a": NaiveExecutionTimeModel()},
    "fitted-naive": {"n/a": FittedNaiveExecutionTimeModel()},
}

In [2]:
# import numpy as np

ttfs = np.linspace(0, 5, 25)
ttfs

array([0.        , 0.20833333, 0.41666667, 0.625     , 0.83333333,
       1.04166667, 1.25      , 1.45833333, 1.66666667, 1.875     ,
       2.08333333, 2.29166667, 2.5       , 2.70833333, 2.91666667,
       3.125     , 3.33333333, 3.54166667, 3.75      , 3.95833333,
       4.16666667, 4.375     , 4.58333333, 4.79166667, 5.        ])

In [3]:
import pandas as pd
import itertools as it

combs = deque()
for model_name, variants in models.items():
    for neuro, model in variants.items():
        for ttf, run in it.product(ttfs, range(1, runs_per_model + 1)):
            combs.append((model, model_name, neuro, ttf, run))

In [4]:
from tqdm.notebook import tqdm_notebook as tqdm

curr_idx = 0
tqdm_iter = tqdm(combs)

rows = deque()

for model, model_name, neuro, ttf, repetition in tqdm_iter:
    tqdm_iter.set_description(
        desc=f"Model {model_name}, neuro {neuro}, TTF {ttf:0.02f}s, repetition {repetition}"
    )
    model.reset()
    cum_duration = 0.0

    for step in range(1, task_steps + 1):
        exec_time = model.advance(ttf).get_execution_time()
        duration = exec_time + ttf
        cum_duration += duration

        rows.append(
            {
                "model": model_name,
                "neuroticism": neuro,
                "repetition": repetition,
                "step": step,
                "exec_time": exec_time,
                "ttf": ttf,
                "duration": duration,
                "cumulative_duration": cum_duration,
            }
        )

        # results.loc[curr_idx, ["model", "neuroticism", "repetition", "step", "exec_time", "ttf", "duration", "cumulative_duration"]] = \
        #     [model_name, neuro, repetition, step, exec_time, ttf, duration, cum_duration]


        # model_name
        # results.at[curr_idx, "neuroticism"] = neuro
        # results.at[curr_idx, "repetition"] = repetition
        # results.at[curr_idx, "step"] = step
        # results.at[curr_idx, "exec_time"] = exec_time
        # results.at[curr_idx, "ttf"] = ttf
        # results.at[curr_idx, "duration"] = duration
        # results.at[curr_idx, "cumulative_duration"] = cum_duration

        curr_idx += 1

results = pd.DataFrame(rows)
results["model"] = results["model"].astype(pd.CategoricalDtype(models.keys(), ordered=False))
results["neuroticism"] = results["neuroticism"].astype(pd.CategoricalDtype(["n/a", "low", "high"], ordered=True))
results["repetition"] = results["repetition"].astype(int)
results["step"] = results["step"].astype(int)
results["exec_time"] = results["exec_time"].astype(float)
results["ttf"] = results["ttf"].astype(float)
results["duration"] = results["duration"].astype(float)
results["cumulative_duration"] = results["cumulative_duration"].astype(float)

results.to_parquet("./timing_models_scaling_ttf.gzip", compression="gzip", partition_cols=["model"])

results

  0%|          | 0/7500 [00:00<?, ?it/s]

,model,neuroticism,repetition,step,exec_time,ttf,duration,cumulative_duration
0,empirical,low,1,1,4.319828,0.0,4.319828,4.319828
1,empirical,low,1,2,5.836220,0.0,5.836220,10.156047
2,empirical,low,1,3,4.386451,0.0,4.386451,14.542498
3,empirical,low,1,4,4.679171,0.0,4.679171,19.221669
4,empirical,low,1,5,4.728985,0.0,4.728985,23.950654
...,...,...,...,...,...,...,...,...
749995,fitted-naive,n/a,30,96,4.827715,5.0,9.827715,1048.463656
749996,fitted-naive,n/a,30,97,3.744538,5.0,8.744538,1057.208194
749997,fitted-naive,n/a,30,98,5.411818,5.0,10.411818,1067.620012
749998,fitted-naive,n/a,30,99,3.957123,5.0,8.957123,1076.577135
